In [3]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 5 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with log tranformation
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [4]:
df = pd.read_csv('../../Punya Andi/UFC_kombinasi5.csv')
df.head()

,B_Height_cms,B_Weight_lbs,B_avg_CLINCH_att,gender,B_win_by_TKO_Doctor_Stoppage,B_avg_LEG_att,B_age,B_win_by_Submission,B_avg_LEG_landed,R_avg_opp_CTRL_time(seconds),...,B_avg_opp_HEAD_landed,B_avg_CTRL_time(seconds),B_avg_opp_TOTAL_STR_landed,B_avg_opp_SUB_ATT,B_avg_opp_GROUND_att,B_avg_KD,R_avg_KD,R_avg_opp_GROUND_att,B_current_lose_streak,B_Reach_cms
0,5.817944,5.707110,1.770706,1,0.0,2.197225,3.806662,0.000000,1.998077,5.326054,...,3.198673,4.892520,4.363099,0.693147,1.998096,0.000000,0.000000,2.442347,0.000000,5.825469
1,5.802722,5.602119,1.897901,1,0.0,2.018023,3.828641,0.000000,1.870600,4.753590,...,3.336937,3.609439,3.528656,0.124703,0.436237,0.768371,0.405465,1.386294,0.693147,5.787265
2,5.817944,5.655992,0.000000,1,0.0,1.609438,3.850148,0.000000,1.098612,4.376726,...,3.167583,2.844182,3.701302,0.000000,0.223144,0.405465,0.000000,2.339399,0.000000,5.825469
3,5.795024,5.707110,2.554680,1,0.0,1.486564,3.850148,0.000000,1.298573,4.764674,...,2.237213,4.238174,3.390200,0.060625,0.594707,0.527867,0.019852,0.230950,0.000000,5.779446
4,5.763622,5.525453,2.395985,1,0.0,2.227707,3.970292,1.386294,1.928921,5.098627,...,3.329550,5.351532,4.281648,0.000244,1.917504,0.405465,0.277868,0.145182,0.693147,5.763622


In [5]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']
y = np.exp(y)

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

# Create the KNN model
knn = KNeighborsRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2')

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)


Best Parameters: {'leaf_size': 10, 'metric': 'manhattan', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Best R-squared Score: 0.7155939794145744


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Evaluate the best model using 5-fold cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='r2')

# Calculate the mean R-squared
mean_r2 = cv_scores.mean()
print("Mean R-squared:", mean_r2)

Mean R-squared: 0.7155939794145744


In [9]:
# Calculate the RMSE using 5-fold cross-validation
cv_rmse = np.sqrt(np.abs(cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')))

# Calculate the mean RMSE
mean_rmse = cv_rmse.mean()
print("Mean RMSE:", mean_rmse)

Mean RMSE: 5.505673315069306


In [10]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi5.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1)
df_test = scaler.transform(df_test)
y_pred = grid_search.best_estimator_.predict(df_test)
submission = pd.DataFrame({'id': df_test_id, 'B_Reach_cms': y_pred})
submission.to_csv('pred_kombinasi4_knn.csv', index=False)